<a href="https://colab.research.google.com/github/shcho11/00.Projects_2024/blob/main/202411_statistical_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **통계적 검정 정리노트**
- revision history : 20241127 / 20240908(init)
- author : 조송현

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
# basic imports

import os
import sys
import time
from datetime import datetime, timedelta
import pandas as pd
import numpy as np
from scipy import stats
import openpyxl
from mlxtend .frequent_patterns import apriori, association_rules
import warnings
warnings.filterwarnings("ignore")
from tqdm import tqdm

In [ ]:
# visualization

import matplotlib.pyplot as plt
from matplotlib import font_manager, rcParams, style

# **통계적 검정 개요**
## 차이 관련 검정
- **평균(Mu)의 차이**
  - 비교대상 모집단 1개 :
    - 모분산을 안다 -> (z) 1표본 z검정
    - 모분산을 모른다 (t) 1표본 t검정 (ttest_1samp)
      - wilcoxon 검정 (정규성불만족시 비모수적 검정)

  - 비교대상 모집단 2개 :
    - 모분산을 안다 -> (z) 2표본 z검정
    - 모분산을 모른다 -> (t) 2표본 t검정 (ttest_ind)
      - 독립표본 t검정 (ex. 남/녀간 평균차이)
        - mannwhitney-u검정 (정규성불만족시 비모수적 검정)
      - 대응표본 t검정 (ex. 1개월뒤, 2개월뒤 반복 측정 case) (ttest_rel)

  - 비교대상 모집단 3개 이상 :
    - ANOVA 분산분석 (statsmodels.stats.anova)
      - 일원 분산분석 (One-way ANOVA) (ex. 한국, 미국, 독일의 평균 키 차이)
        - 반복측정 분산분석 (Repeated Measures ANOVA) (1개월뒤, 2개월뒤 반복 측정 case)
      - 이원 분산분석 (Two-way ANOVA) (2개 독립변수가 종속변수에 미치는 영향과 상호작용을 동시에 분석할 때)
        - 이원 반복측정 분산분석 (Two-way Repeated Measures ANOVA)

- **분산(분포)의 차이**
  - F 검정 (분산 차이를 검정) (분포가 정규성을 만족할 경우를 전제)
  - Levene's test (정규성을 만족하지 않을 때도 사용 가능)
  - Bartlett

<br>

## 관계 관련 검정
- **범주형 확률 분포의 적합도 및 독립성**
  - chi-square(카이제곱) 검정 (기대분포와의 적합도) (chisquare)
  - chi2_contingency 검정 (독립성 검정, 2d) (chi2_contingency)

- **범주형 변수 간 관계 분석**
  - 교차분석 (범주형자료 A가 또 다른 범주형자료 B에 영향을 주는 지를 확인)
- **수치형 변수 간 관계 분석**
  - 상관분석(Correlation)
- **수치형 변수 간 종속관계(인과관계) 분석**
  - 회귀분석(Regression)
    - 독립변수 : 종속변수 관계가 1:1 -> 단순회귀분석
    - 독립변수 : 종속변수 관게가 N:1 -> 다중회귀분석
    - 로지스틱회귀분석 (분류) (statsmodels.formula.api.logit)
      - odds ratio분석 (np.exp(md.params)

In [ ]:
from sklearn.datasets import load_iris

iris = load_iris()
df_iris = pd.DataFrame(iris.data, columns = iris.feature_names)
df_iris['target'] = iris.target
df_iris.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0


## 기술통계량

In [ ]:
sepal_length = df_iris['sepal length (cm)']

print("평균 : ", np.mean(sepal_length))
print("분산 : ", np.var(sepal_length))
print("표준편차 : ", np.std(sepal_length))
print("최대값 : ", np.max(sepal_length))
print("최소값 : ", np.min(sepal_length))
print("중앙값 : ", np.median(sepal_length))

print("백분위(25) : ", np.percentile(sepal_length,25))
print("사분위(1) : ", np.quantile(sepal_length,0.25))

평균 :  5.843333333333334
분산 :  0.6811222222222222
표준편차 :  0.8253012917851409
최대값 :  7.9
최소값 :  4.3
중앙값 :  5.8
백분위(25) :  5.1
사분위(1) :  5.1


## 베르누이 분포 통계적 검정 (이항검정; Binominal Test)
- stats.bernoulli(mu).rvs(N) -> np.count_nonzero() -> stats.binomtest(n, N)


In [ ]:
N1 = 10
N2 = 100
N3 = 1000

mu = 0.5
np.random.seed(0)

x1 = stats.bernoulli(mu).rvs(N1)
x2 = stats.bernoulli(mu).rvs(N2)
x3 = stats.bernoulli(mu).rvs(N3)

n1 = np.count_nonzero(x1)
n2 = np.count_nonzero(x2)
n3 = np.count_nonzero(x3)

# 유의확률

print(stats.binomtest(n1, N1))
print(stats.binomtest(n2, N2))
print(stats.binomtest(n3, N3))

BinomTestResult(k=7, n=10, alternative='two-sided', statistic=0.7, pvalue=0.34375)
BinomTestResult(k=49, n=100, alternative='two-sided', statistic=0.49, pvalue=0.9204107626128211)
BinomTestResult(k=481, n=1000, alternative='two-sided', statistic=0.481, pvalue=0.24196933616887067)


- 베르누이 확률분포의 모수가 0.5라는 귀무가설 하에 위 가설은 통계적으로 유의하지 않다

# **집단의(집단 간) 평균 차이 검정 (z검정, t검정)**

# **1) 단일표본 z 검정**
- 모집단의 표준편차(모표준편차)가 알려져 있는 경우를 전제
- **검정의 목적 : 한 그룹(표본)의 평균이 특정 기대값(모평균)과 유의미하게 다른 지를 검정**
  - ex) 어떤 학교 학생들의 평균키가 175cm인지 검정하기 위해, 해당 학교의 임의선택된 학생들의 평균 키를 조사
- 잘 사용되지 않음(scipy에도 별도 함수 X) / norm명령어, cdf메서드 사용하여 직접 구현

In [ ]:
# 실제 모수 mu=0, var=1인 경우, 표본데이터 N을 시뮬레이션으로 구하여 귀무가설 mu=0에 대한 단일표본 z검정

# N=10 인 경우
N=10
mu_0 = 0
np.random.seed(0)

x = stats.norm(mu_0).rvs(N)
#print(x)

In [ ]:
print(np.mean(x))
print(np.var(x))
print(np.std(x))

0.7380231707288347
0.9352422005002838
0.9670792110785361


In [ ]:
def ztest_1samp(x, sigma2=1, mu=0) :
  z = (x.mean()-mu) / np.sqrt(sigma2/len(x)) # z-통계량 계산
  return z, 2*stats.norm().sf(np.abs(z)) # z와 p-value 반환

ztest_1samp(x)

(2.3338341854824276, 0.019604406021683538)

- 유의확률 0.019 따라서 귀무가설을 기각(mu<>0)
- 그러나 이 경우 데이터수가 10개로 절대적 부족, 귀무가설이 참임에도 기각된 경우(Type1 Error)인 가능성을 배제할 수 없음.

In [ ]:
# N=100 인 경우

N=100
mu_0 = 0
np.random.seed(0)
x100 = stats.norm(mu_0).rvs(N)
ztest_1samp(x100)

(0.5980801553448499, 0.5497864508624168)

- 이처럼 N을 100으로 설정하니 유의확률 0.549 즉 귀무가설을 기각할 수 없다 (mu=0 이다)

# **2) 단일표본 t검정**
- 모집단의 표준편차가 알려져 있지 않은 경우 & 데이터가 정규분포를 따르거나 표본크기가 충분히 큰 경우를 전제
- **한 그룹의 평균이 특정 기대값(모평균)과 유의미하게 다른지 검정**
  - ex) 샘플로부터 추출된 제품의 평균 무게가 500g 인지 검정
- scipy ttest_1samp 사용 (디폴트 모수가 없으므로 기대값(popmean) 인수를 직접 지정해야함
- scipy.stats.ttest_1samp(a,popmean) # a : 표본데이터 배열, popmean: 귀무가설의 기대값

In [ ]:
# 데이터갯수 N=100, 실제모수 mu=0인 경우 단일표본 t 검정

N=100
mu_0 = 0
np.random.seed(0)
x = stats.norm(loc=mu_0).rvs(N)

stats.ttest_1samp(x, popmean=0)

TtestResult(statistic=0.5904283402851698, pvalue=0.5562489158694675, df=99)

- 유의확률 0.556 따라서 귀무가설을 기각할 수 없다 (mu=0 이다)

# **독립표본 t 검정 (scipy ttest_ind)**
- **2개의 독립적인 정규분포에서 나온 N1개, N2개 데이터셋을 이용, 두 정규분포의 기댓값이 동일한한지 여부를 검사**
  - ex) 대면채널 관리고객과 비대면채널 관리고객의 펀드 평균가가입 금액에 차차이가 있다고 가정시 이를 검정

- 프로세스
1. **데이터의 정규성 검정**
  - shapiro 정규성 검정 수행
  - 만약 정규성을 만족하지 못할 경우: 독립표본 t 검정의 대안으로 'Mann-Whitney U 검정'을 수행 (그럼에도 표본 크기 30개 이상으로 충분히 크거나, 대략적인 정규성을 갖는다면 t 검정을 사용할 순 있음. 단 이 경우 해석에 주의 필요)
  - - stats.shapiro(df['x'])
2. **두 정규분포의 분산 값 일치 여부 (등분산성)**
  - 독립표본 t 검정은 두 정규분포의 분산 값이 같은 경우/같지 않은 경우 검정통계량이 다름. 따라서 equal_var 인수를 사용하여 지정해줘야 함
  - 두 정규분포의 분산 값이 같은지 여부는 등분산검정(equal-variance test)를 사용 (잘 모르겠으면 equal_var=False로 설정)
3. **독립표본 t검정 수행**
  - ttest_ind(sample1, sample2, alternative, equal_var=FALSE)
    - alternative = 'two-sided', 'less', 'greater'
    - eqaul_var = TRUE(등분산성 만족) FALSE(등분산성 만족하지않음)

In [ ]:
# 독립표본 t 검정

# 난수 갯수
N_1 = 50
N_2 = 100

# 평균
mu_1 = 0
mu_2 = 0.5

# 표준편차
sigma_1 = 1
sigma_2 = 2

np.random.seed(0)

x1 = stats.norm(mu_1, sigma_1).rvs(N_1)
x2 = stats.norm(mu_2, sigma_2).rvs(N_2)

stats.ttest_ind(x1, x2, equal_var=False)

TtestResult(statistic=-2.3642776009118034, pvalue=0.01939873896502667, df=144.13359611582456)

- p-value가 0.019 따라서 "두 확률분포의 기댓값이 일치한다"라는 귀무가설을 기각할 수 있음 ("일치하지 않는다")

# **단측 독립표본 t 검정**
- 한 집단의 평균이 다른 집단의 평균보다 큰 지를 검정
- 단측 검정에서는 가설이 특정 방향성을 가지며, 귀무가설을 기각하는 기준이 양측이 아닌 단측 방향으로 설정
  - H0: 집단 1의 평균이 집단 2의 평균보다 작거나 같다
  - H1: 집단 1의 평균이 집단 2의 평균보다 크다

In [ ]:
# 방법 1 : 간단하게 alternative 사용

group1 = np.random.normal(12,2,30)
group2 = np.random.normal(10,2,30)

stats.ttest_ind(group1, group2, equal_var=False, alternative='greater')

TtestResult(statistic=6.632720358793076, pvalue=6.925861047340181e-09, df=56.11697288097011)

In [ ]:
# 방법 2 : 논리적으로 풀어서

from scipy.stats import ttest_ind

def ttest_one_sided(group1, group2, group1name, group2name, alpha) :

  # 두 그룹 데이터 생성
  np.random.seed(0)

  # t test 수행
  t_stat, p_value = ttest_ind(group1, group2, equal_var=False)

  # 단측 검정
  if t_stat > 0 :
    p_value_onesided = p_value/2

    print(f"t_stat : {t_stat} \np_value_onesided : {p_value_onesided}")

    # 결과 해석
    if p_value_onesided < alpha :
      print(f"독립표본 t 단측검정 결과 {group1name} 평균이 {group2name} 평균 대비 통계적으로 유의하게 크다 \n")

    else :
      print(f"독립표본 t 단측검정 결과 {group1name} 평균이 {group2name} 평균 대비 통계적으로 유의하게 크지 않다 \n")

  else : # 태세전환하여 오히려 반대의 경우를 검정해본다

    t_stat, p_value = ttest_ind(group2, group1)
    p_value_onesided = p_value/2

    print(f"t_stat : {t_stat} \np_value_onesided : {p_value_onesided}")

    if p_value_onesided < alpha :
      print(f"!! {group1name} 평균이 {group1name} 평균 대비 크지 않다 !!")
      print(f"!! 오히려 {group1name} 평균이 {group2name} 평균 대비 통계적으로 유의미하게 작다 !! \n")

    else :
      print(f"!! {group1name} 평균이 {group2name} 평균 대비 크지 않다 !!")
      print(f"단, 역방향의 단측 검정 결과 또한 통계적으로 유의하지 않다 \n")

ttest_one_sided(group1, group2, "그룹1", "그룹2", 0.05)

t_stat : 6.632720358793076 
p_value_onesided : 6.925861047340181e-09
독립표본 t 단측검정 결과 그룹1 평균이 그룹2 평균 대비 통계적으로 유의하게 크다 



In [ ]:
group3 = np.random.normal(10,2,30)
group4 = np.random.normal(13,2,30)

ttest_one_sided(group3, group4, "그룹3", "그룹4", 0.05)

t_stat : 2.9389770379445195 
p_value_onesided : 0.002360167719122182
!! 그룹3 평균이 그룹3 평균 대비 크지 않다 !!
!! 오히려 그룹3 평균이 그룹4 평균 대비 통계적으로 유의미하게 작다 !! 



# 일원 분산분석(One-way ANOVA)을 통한 평균 차이 검정

In [ ]:
import numpy as np
from scipy.stats import f_oneway

# 두 그룹 데이터 생성
np.random.seed(0)
group1 = np.random.normal(10,3,30)
group2 = np.random.normal(10,2,30)
group3 = np.random.normal(10,4,30)

# 일원 분산분석
f_stat, p_value = f_oneway(group1, group2, group3)

print(f"F-stat : {f_stat} \n p_value : {p_value}")

# 결과 해석
alpha = 0.05
if p_value < alpha :
  print("일원 분산분석 결과 두 그룹 평균은 유의미하게 차이가 있다.")
else :
  print("일원 분산분석 결과 두 그룹 평균은 유의미하게 차이가 있지 않다.")

F-stat : 3.661566870753982 
 p_value : 0.029729997403484455
일원 분산분석 결과 두 그룹 평균은 유의미하게 차이가 있다.


# **대응표본 t검정(쌍체표본 t검정)**
- 독립표본 t검정을 두 집단의 표본이 1대1 대응하는 경우에 대해 수정한 것. (시점or조건에 따라)
- 즉, 독립표본 t검정과 마찬가지로 두 정규분포의 기대값이 동일한지 확인하기 위한 검정
  - stats.ttest_rel(x,y,alternative)
    - x : 처리방법이 x일 때 관측값
    - y : 처리방법이 y일 때 관측값
    - alternative = 'two-sided', 'less', 'greater'

In [ ]:
# 7명의 환자 대상. 수면영양제 복용 전 후 수면시간 측정하여 영양제 효과 측정
# 유의수준 0.05

# 복용 전 수면시간 [5,3,8,4,3,2,1]
# 복용 후 수면시간 [8,6,6,5,8,7,3]

# 귀무 : 수면시간 차이없음 /  대립 : 수면시간 늘어남

import pandas as pd
from scipy.stats import ttest_rel

data = pd.DataFrame({'before' : [5,3,8,4,3,2,1],
                     'after' : [8,6,6,5,8,7,3]})

result = ttest_rel(data['after'], data['before'], alternative='greater') # after가 before보다 커졌는지 검정

print('t검정통계량 = %.3f, pvalue = %.3f' %(result))

t검정통계량 = 2.634, pvalue = 0.019


- p = 0.019 (< 0.05) 즉 귀무가설을 기각, 대립가설 채택 (수면영양제 복용 후 수면시간이 유의미하게 늘었음)


# **두 집단 간 분산 비교 검정 (F-검정, Levene's, Bartlett)**
- 두 집단 간의 분산이 동일한지 여부를 판단하기 위한 고전적인 방법
- Levene's test를 이용하면 분포가 정규성을 만족하지 않더라도 사용 가능
  - e.g. 두 그룹의 수익률 변동성(분산)이 동일한지 비교
- bartlett의 경우 각 데이터가 정규성 만족해야 함
- F test 공식 : F = s1^2 / s2^2 (단, s1^2 > s2^2)

In [ ]:
import numpy as np

np.random.seed(0)
group1 = np.random.normal(10, 3, 1000)
group2 = np.random.normal(10, 2, 1000)

In [ ]:
# shapiro 를 이용하여 정규성 여부 확인

from scipy.stats import shapiro

stat1, pv1 = shapiro(group1)
stat2, pv2 = shapiro(group2)

print(f"Group1 : Test Statistic: {stat1:.3f}, p-value: {pv1:.3f}")
print(f"Group2 : Test Statistic: {stat2:.3f}, p-value: {pv2:.3f}")

Group1 : Test Statistic: 0.999, p-value: 0.591
Group2 : Test Statistic: 0.999, p-value: 0.740


In [ ]:
# Levene 검정

from scipy.stats import levene

levene_stat, p_value2 = levene(group1, group2)

print(f"F-statistic : {levene_stat}, p-value : {p_value2}")

F-statistic : 143.6073040779633, p-value : 5.262451654071628e-32


In [ ]:
# 정규분포 가정시

# F-검정

from scipy.stats import f

def f_test(x, y) :
  if np.var(x, ddof=1) < np.var(y, ddof=1) :
    x, y = y, x

  f_stat = np.var(x, ddof=1) / np.var(y, ddof=1)

  x_dof = x.size - 1
  y_dof = y.size - 1

  p_value = (1-f.cdf(f_stat, x_dof, y_dof)) * 2
  return f_stat, p_value

f_stat, p_value1 = f_test(group1, group2)
print(f"F-statistic : {f_stat}, p-value : {p_value1}")

F-statistic : 2.3385773921900666, p-value : 2.220446049250313e-16


In [ ]:
# 정규분포 가정시

# Bartlett 검정

from scipy.stats import bartlett

bartlett_stat, p_value3 = bartlett(group1, group2)

print(f"F-statistic : {bartlett_stat}, p-value : {p_value3}")

F-statistic : 174.98958319673073, p-value : 6.01840545249289e-40


# **카이제곱 적합도 검정 (Chi-square Goodness-of-Fit test)**
- 범주형 자료 간 차이 보여주는 분석 방법
- 관찰된 빈도가 기대되는 빈도와 유의하게 다른지 검정하는 방법.

  - 귀무가설 : 표본집단의 분포가 주어진 특정 분포를 따른다
  - 관찰빈도와 기대빈도의 차이가 클수록 귀무가설 기각할 확률이 커짐
- chisquare(f_obs, f_exp) # f_obs : 관찰 빈도 나타내는 데이터, f_exp : 기대 빈도를 나타내는 데이터

In [ ]:
# 유의수준 0.05. 남학생 90명 여학생 160명 있는데 남,여학생 비율이 45:55 인지 카이제곱 검정을 통해 분석.
# 귀무 : 비율이 45:55 / 대립 : 비율이 45:55가 아님

import numpy as np
from scipy.stats import chisquare

num = np.array([90, 160])
exp = np.array([0.45,0.55]) * np.sum(num)

chi2, pv = chisquare(num, f_exp = exp)
print(f"Chi2 Statistic : {chi2}, p-value : {pv}")

Chi2 Statistic : 8.181818181818182, p-value : 0.004231232899758152


- p value < 0.05 따라서 귀무가설 기각. 대립가설 (45:55가 아님)을 채택

# **카이제곱 독립 검정 (Chi-square test for Independence)**
- 변수가 두 개 이상 범주로 분할되어 있을 때 , 각 범주가 서로 독립적인지 검정
- 귀무 : 독립적이다
  - 독립성 검정함수 : chi2_contingency(observed)
- observed : 두 개 이상 변수를 포함하는 2차원 배열

In [ ]:
# survey 데이터셋 에서 성별에 따른 운동 빈도가 관계가 있는지 카이제곱 검정을 이용하여 분석.

import pandas as pd
from scipy.stats import chi2_contingency

survey = pd.read_csv('drive/MyDrive/soojebi/survey.csv')

tb = pd.crosstab(survey['Sex'], survey['Exer'])
print(tb)

Exer    Freq  Some
Sex               
Female    49    58
Male      65    40


In [ ]:
chi2, pv, dof, expected = chi2_contingency(tb)

print(f"Chi2 Statistic : {chi2}, p-value : {pv}")
print(f"Deg of freedom : {dof}, \n Expected frequencies Table : {expected}")

Chi2 Statistic : 4.904232352768243, p-value : 0.0267909570897706
Deg of freedom : 1, 
 Expected frequencies Table : [[57.53773585 49.46226415]
 [56.46226415 48.53773585]]


- p = 0.02 따라서 귀무가설을 기각 (즉, 성별에 따른 운동빈도는 통계적으로 유의한 상관관계가 있다)